# Wrapper Method
<br> Wrapper Method Process <br>
<br><img src = 'img/WrapperMethod.png' width="40%" height="40%"/> 

<br> 보는 바와 같이 Wrapper Method 는 features 를 선정하는 과정에서 반복 작업이 일어납니다. 
<br> 이 과정에서 자신을 계속해서 사용하기 때문에 Recursive(재귀)<sup>[1](#footnote_1)</sup>라는 말을 사용한 것으로 판단됩니다.
<br> 저는 Wrapper Method 중 Recursive Feature Elimination 을 코드로 구현해보았습니다.

## 코드를 짜면서 깨달은 점

<br>1. Wrapper Method 는 단순히 Ranking 에 기반한 Features Selection 이 아니다. <br><br> 즉, n feature select 늘릴 때 이전에 포함되었던 feature 가 반드시 포함될 수 없다는 뜻이다. <br> 다시 말하면, 떡볶이와 함께 먹을 음식 1가지를 고르세요 했을 때는 튀김이 선택됐지만 <br> 2가지를 고르라고 했을 때는 주먹밥, 순대가 선택될 수 있다는 의미이다. <br> 수정, Decision Tree 에서만 보이는 현상인 것 같다. <br>

<br>2. 반복작업이 일어난다. 그 말은 즉, 처리시간이 오래 걸린다.<br><br> Variance Selection, RF-importance 를 사용했을 때 보다 느린 속도를 보여줌<br>estimator 에 따라서도 속도 차이를 보인다.<br>


<br><a name="footnote_1">1</a>. 사전적 의미 : 자신을 정의할 때 자기 자신을 재참조한다. 
<br> 반복과정에서 자기 자신(아마도, Features를 재참조한다는 의미인듯) 을 다시 사용한다. 재귀에 대한 이해가 아직 되지 않았다면 factorial 함수를 생각해봅시다. 10000! 를 계산해보라는 문제가 있습니다. 
<br> 우리는 이 문제를 해결하기 위해 1,2,3,4 ~ 9999,10000 을 차례로 곱했을 것입니다. 사실 이게 재귀나 다름없지만 조금 다르게 접근해봅시다. <br>
<br> 10000! = 10000 X 9999! 입니다. 다시, 10000! = 10000 X 9999 X 9998! 입니다. <br>
<br> 우리는 10000! 에서 10000을 제외하고 9999! 을 다시 참조합니다. 다음은 9999를 떼어 9998! 을 참조합니다. 연산이 마지막까지 도달하면 1! 을 참조할 것입니다. 
<br> 이렇게 자기 자신의 일부? 혹은 전체를 참고하는 것을 재귀라고 합니다.

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('../mushrooms.csv')

### RFE(Recursive Feature Elimination 사용 방법)

In [18]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X, y)
selector.support_ # bool type 으로 반환

array([ True,  True,  True,  True,  True, False, False, False, False,
       False])

In [19]:
selector.ranking_ # 순위를 매겨줌 True 는 모두 1 임


array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

### Mushrooms data 에서 적용

In [20]:
dataset_features = dataset.drop('class', axis=1)
dataset_target = dataset['class']

dataset_features.shape

(8124, 22)

In [21]:
train_features, test_features, train_target, test_target = train_test_split(dataset_features, dataset_target, test_size=0.5, random_state=3) # size 를 0.5 로 둔것은 생각보다 오래걸려서

In [22]:
train_features = pd.get_dummies(train_features)
test_features = pd.get_dummies(test_features)

print(train_features.shape, test_features.shape)

(4062, 117) (4062, 117)


In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

## Estimator = RF Classifier

In [24]:
model = RandomForestClassifier()
selector = RFE(model, n_features_to_select=8, step=1) # step 단계마다 제거할 feature 으 갯수 만약 1보다 작다면 제거할 feature의 백분률율에 해당
selector = selector.fit(train_features, train_target)
train_features.columns[selector.get_support()]

Index(['odor_f', 'odor_n', 'gill-size_b', 'gill-size_n', 'gill-color_b',
       'stalk-shape_t', 'stalk-surface-above-ring_k', 'ring-type_p'],
      dtype='object')

In [25]:
selector.ranking_

array([ 46,  88,  63,  86,  90,  52,  37, 106,  29,  50,  48,  66,  93,
        61,  59,  47,  79,  85,  44,  36,   6,   2,  24,  15,   1,  26,
        78,   1,   9,  68,  42,  95,  83,   8,  12,   1,   1,   1,  92,
        54,  72,  84,  64, 100,  89,  51,  71,  43,  98,  11,   1,  23,
         5,  16,  14,  76,  41,   1,  19,  74,  25,   3,  32,  22, 102,
        73, 104,  97,  96,  75,  82,  33,  80,  87,  53, 103,  94,  58,
        91,  70,  40,  56, 108, 107,  99,  65,  77,  60,  27,  17,  28,
        49,  21,  62,   1, 109,   4,  35,  34, 101,   7,  69,  10, 110,
       105,  45,  57,  38,  13,  31,  20,  30,  67,  39,  55,  18,  81])

In [30]:
model = RandomForestClassifier()
selector = RFE(model, n_features_to_select=9, step=1) # 선택할 features 를 8개에서 9개로 변경
selector = selector.fit(train_features, train_target)
train_features.columns[selector.get_support()] # stalk-shape-t 가 없어지고 stalk-surface-below-ring_k, spore-print-color_h 생김

Index(['odor_f', 'odor_n', 'gill-size_b', 'gill-size_n', 'gill-color_b',
       'stalk-surface-above-ring_k', 'stalk-surface-below-ring_k',
       'ring-type_p', 'spore-print-color_h'],
      dtype='object')

## estimator = SVC

In [31]:
for i in range(1,10):
    model = SVC(kernel='linear') # kernel : {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} 아쉬운 점은 linear 밖에 안됨 kernel's default = rbf 이다.
    selector = RFE(model, n_features_to_select= i, step=1)
    selector = selector.fit(train_features, train_target)
    print('feature 를', i,'개 선택할 때')
    print(train_features.columns[selector.get_support()])
    print('------------ 구분선---------------')

feature 를 1 개 선택할 때
Index(['odor_l'], dtype='object')
------------ 구분선---------------
feature 를 2 개 선택할 때
Index(['odor_a', 'odor_l'], dtype='object')
------------ 구분선---------------
feature 를 3 개 선택할 때
Index(['odor_a', 'odor_l', 'spore-print-color_r'], dtype='object')
------------ 구분선---------------
feature 를 4 개 선택할 때
Index(['odor_a', 'odor_l', 'stalk-surface-below-ring_y',
       'spore-print-color_r'],
      dtype='object')
------------ 구분선---------------
feature 를 5 개 선택할 때
Index(['odor_a', 'odor_l', 'odor_n', 'stalk-surface-below-ring_y',
       'spore-print-color_r'],
      dtype='object')
------------ 구분선---------------
feature 를 6 개 선택할 때
Index(['odor_a', 'odor_l', 'odor_n', 'gill-size_n',
       'stalk-surface-below-ring_y', 'spore-print-color_r'],
      dtype='object')
------------ 구분선---------------
feature 를 7 개 선택할 때
Index(['odor_a', 'odor_f', 'odor_l', 'odor_n', 'gill-size_n',
       'stalk-surface-below-ring_y', 'spore-print-color_r'],
      dtype='object')
------------ 

# 흥미로운 점
### model, 다시 말해 estimator 를 RF 로 사용했을 때와 SVC 로 사용했을 때의 선택되는 Features 의 차이가 생각보다 크다는 것